# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet
#tfdet.core.util.set_seed(777) #set seed
device = tfdet.core.util.select_device(0) #set device

# 1. Init Dataset

In [3]:
import numpy as np

image_shape = [32, 32]
label = ["OK", "NG"]
train_size = 1000
val_size = 100
test_size = 100
batch_size = 16

def preprocess(x_true, y_true = None):
    x_true = np.expand_dims(x_true, axis = -1)
    x_true = np.tile(x_true, [1, 1, 3])
    if y_true is None:
        return x_true
    else:
        return x_true, y_true
    
(tr_x, tr_y), (te_x, te_y) = tf.keras.datasets.mnist.load_data()

tr_dataset = tfdet.dataset.Dataset(tr_x[tr_y == 0][:train_size],
                                   transform = [preprocess, #for mnist (28, 28) > (28, 28, 3)
                                                {"name":"load"}, #image is file
                                                {"name":"resize", "image_shape":image_shape},
                                                {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]},
                                                {"name":"pad", "image_shape":image_shape, "pad_val":0}])

val_y = np.expand_dims((te_y[:val_size] != 0).astype(np.int32), axis = -1) #0 is ok, etc is ng.
val_dataset = tfdet.dataset.Dataset(te_x[:val_size],
                                    transform = [preprocess, #for mnist (28, 28) > (28, 28, 3)
                                                 {"name":"load"}, #image is file
                                                 {"name":"resize", "image_shape":image_shape},
                                                 {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]},
                                                 {"name":"pad", "image_shape":image_shape, "pad_val":0}])

te_y = np.expand_dims((te_y[val_size:val_size + test_size] != 0).astype(np.int32), axis = -1) #0 is ok, etc is ng.
te_dataset = tfdet.dataset.Dataset(te_x[val_size:val_size + test_size],
                                   transform = [preprocess, #for mnist (28, 28) > (28, 28, 3)
                                                {"name":"load"}, #image is file
                                                {"name":"resize", "image_shape":image_shape},
                                                {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]},
                                                {"name":"pad", "image_shape":image_shape, "pad_val":0}])

tr_pipe = tfdet.dataset.PipeLoader(tr_dataset, batch_size = batch_size, prefetch = True)
val_pipe = tfdet.dataset.PipeLoader(val_dataset, batch_size = batch_size, prefetch = True)
te_pipe = tfdet.dataset.PipeLoader(te_dataset, batch_size = batch_size, prefetch = True)

# 2. Build Detector

In [4]:
with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3])
    out = tfdet.model.backbone.wide_resnet50_2(x, weights = "imagenet_v2", indices = [0, 1, 2])
    model = tf.keras.Model(x, out)
    feature = model.predict(tr_pipe, verbose = 1) #feature extract

63/63 [==============================] - 3s 34ms/step


# 3. Train

3-1. Init HyperParameter

In [5]:
sampling_size = 550
memory_reduce = True

n_feature = np.sum([np.shape(f)[-1] for f in feature])
sampling_index = np.random.choice(np.arange(n_feature), sampling_size, replace = False)

3-2. Generate Feature Vector

In [6]:
with device:
    feature_vector = tfdet.model.train.padim.train(feature, sampling_index = sampling_index, memory_reduce = memory_reduce) #memory_reduce is a tradeoff between accuracy and memory

3-3. Build Predict Model

In [7]:
with device:
    out = tfdet.model.detector.padim(out, feature_vector, image_shape = image_shape, sampling_index = sampling_index, memory_reduce = memory_reduce) #align memory_reduce with train in test
    model = tf.keras.Model(x, out)
    score_pred, mask_pred = model.predict(val_pipe, verbose = 1)
    threshold = tfdet.util.get_threshold(val_y, score_pred)
    filtered_out = tfdet.model.postprocess.padim.FilterDetection(threshold = threshold)(out)
    model = tf.keras.Model(x, filtered_out)

7/7 [==============================] - 1s 43ms/step


# 4. Evaluate

In [8]:
score_pred, mask_pred = model.predict(te_pipe, verbose = 0)
print("score : {0:.4f}".format(np.mean((0 < score_pred) == te_y)))

score : 0.9900


# 5. Save & Load

5-1. Save

In [9]:
import os, shutil, pickle

save_path = "./learn/model.pickle"

if os.path.exists(os.path.dirname(save_path)):
    shutil.rmtree(os.path.dirname(save_path))
os.makedirs(os.path.dirname(save_path), exist_ok = True)

tfdet.dataset.util.save_pickle([image_shape, feature_vector, sampling_index, memory_reduce, threshold], save_path)

'./learn/model.pickle'

5-2. Load

In [10]:
save_path = "./learn/model.pickle"

image_shape, feature_vector, sampling_index, memory_reduce, threshold = tfdet.dataset.util.load_pickle(save_path)
    
with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3])
    out = tfdet.model.backbone.wide_resnet50_2(x, weights = "imagenet_v2", indices = [0, 1, 2])
    score, mask = tfdet.model.detector.padim(out, feature_vector, image_shape = image_shape, sampling_index = sampling_index, memory_reduce = memory_reduce) #align memory_reduce with train in test
    filtered_out = tfdet.model.postprocess.padim.FilterDetection(threshold = threshold)([score, mask])
    model = tf.keras.Model(x, filtered_out)
    
score_pred, mask_pred = model.predict(te_pipe, verbose = 0)
print("score : {0:.4f}".format(np.mean((0 < score_pred) == te_y)))

score : 0.9900


# 6. Visualize

In [ ]:
import matplotlib.pyplot as plt

index = 0
x_true = te_dataset[index] #or next(iter(tr_dataset))
y_true = te_y[index]

score_pred, mask_pred = model.predict(np.expand_dims(x_true, axis = 0), verbose = 0)

x_true = tfdet.dataset.transform.unnormalize(x_true, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375])

print(y_true, score_pred[0])
fig_size = (5, 5)
plt.figure(figsize = fig_size)
plt.imshow(x_true)
plt.figure(figsize = fig_size)
plt.imshow(mask_pred[0])